In [1]:
from fdh_gallica import Periodical, Search, Document
from fdh_gallica.parallel_process import iiif_urls_for_documents
from tqdm.autonotebook import tqdm
import json
import pandas as pd
from pandas.io.json import json_normalize
import itertools
from itertools import chain
from itertools import compress
import re
import wikipedia
from bs4 import BeautifulSoup
import requests
from SPARQLWrapper import SPARQLWrapper, XML
import numpy as np

/Users/lia/miniconda3/lib/python3.6/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


ModuleNotFoundError: No module named 'SPARQLWrapper'

In [2]:
# get bnf table using bnf link

def get_bnf_table(link):
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        s = soup.find('table')
        table_rows = s.find_all('tr')

        table = {}
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            if len(row) == 2:
                table[row[0]] = row[1]
            else:
                table[row[0]] = row[1:]
        return table

    except:
        return -1

In [3]:
bnf_summary = pd.read_pickle('data/bnf_summaries.pkl')

In [4]:
bnf_summary.bnf_link

0       http://data.bnf.fr/ark:/12148/cb15581791p#about
1       http://data.bnf.fr/ark:/12148/cb16295857k#about
2       http://data.bnf.fr/ark:/12148/cb122027126#about
3       http://data.bnf.fr/ark:/12148/cb162959137#about
4       http://data.bnf.fr/ark:/12148/cb13945821b#about
                             ...                       
1926    http://data.bnf.fr/ark:/12148/cb11901746b#about
1927    http://data.bnf.fr/ark:/12148/cb119865877#about
1928    http://data.bnf.fr/ark:/12148/cb130148468#about
1929    http://data.bnf.fr/ark:/12148/cb14656875b#about
1930    http://data.bnf.fr/ark:/12148/cb13517018d#about
Name: bnf_link, Length: 1874, dtype: object

In [9]:
bnf_table_ = bnf_summary.bnf_link.apply(get_bnf_table)
bnf_table = bnf_table_.apply(pd.Series)
print('rows missing : {0}'.format(bnf_table.shape[0] - bnf_summary.shape[0]))

rows missing : 0


In [13]:
bnf_table.columns.tolist()

[0,
 '\n\n\n\n',
 'Activité commerciale\xa0:',
 'Autre forme du nom\xa0:',
 'Autres formes du nom\xa0:',
 'Autres identités\xa0:',
 'Domaines\xa0:',
 'ISNI\xa0:',
 'Information (catalogue BnF archives et manuscrits)\xa0:',
 'Langue\xa0:',
 'Mort\xa0:',
 'Naissance\xa0:',
 'Note\xa0:',
 'Pays\xa0:',
 'Sexe\xa0:',
 'Site Web\xa0:']

In [14]:
# rename columns
# may have to fix this after re-running the crawling.
bnf_table.columns = ['null1','null2','activity','othername1','othername2','domain','othername3',\
                     'ISNI','information','lang','died','born','note','country','gender','website']

In [15]:
# check rows without gender

print('# rows w/o gender: {0}'.format(bnf_table[bnf_table.gender.isna()].shape[0]))

# rows w/o gender: 68


In [16]:
# check note field for queers.

bnf_table[bnf_table.gender.isna() & (~bnf_table.note.isna())].note.unique().tolist()

['Soprano américaine (9 décembre 1850 - 5 janvier 1891)',
 'Actrice',
 'Artiste lyrique (soprano)',
 'Linguiste',
 'Artiste de variétés',
 'Comédienne américaine',
 'Ministre de la Marine impériale russe de 1903 à 1905',
 'Baryton et professeur de chant français, né à Paris le 30 juin 1865 et mort le 27 septembre 1907',
 "Ouvrier typographe, avocat, représentant du peuple à la Constituante de 1848 et à la Législative de 1849. - Militant républicain et socialiste. - ″Plusieurs dictionnaires donnent à Martin Bernard les prénoms d'Aristide-Martin. C'est une erreur que l'examen de l'acte de naissance permet de redresser. Plus tard, le nom de Bernard étant très répandu, l'habitude se prendra de l'appeler Martin-Bernard″ : C. Latta, p. 12, n. 2.",
 'Peintre et graveur',
 "Fille du roi François Ier des Deux-Siciles et de l'infante Marie-Isabelle d'Espagne, elle est princesse de la Maison de Bourbon puis devient par son mariage avec Pierre II du Brésil, la troisième et dernière impératrice du 

In [17]:
# seems we can tell their gender with information in note.

bnf_table[bnf_table.gender.isna() & (~bnf_table.note.isna())].note.apply(lambda x: x.split()[0]).unique().tolist()
print("gender can be fixed by looking only at the first word of 'note'")

gender can be fixed by looking only at the first word of 'note'


In [18]:
female = ['soprano','fille','danseuse','actrice','cantatrice','comédienne']

def fix_gender(x):
    if x.split()[0].lower() in female:
        return 'féminin'
    else:
        return 'masculin'

In [19]:
_fix_gender_index = bnf_table.gender.isna() & (~bnf_table.note.isna())

bnf_table['gender'][_fix_gender_index] = bnf_table[_fix_gender_index].note.apply(fix_gender)

C:\Users\haeeun\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
bnf_table[['name','bnf_link']] = bnf_summary[['raw_name','bnf_link']]
bnf_table = bnf_table[['name', 'bnf_link','gender','note','country', 'lang', 'born', 'died','domain',
       'othernames1', 'isni', 'othernames2', 'othernames3', 'activity',
       'information', 'null1','null2', 'website', ]]

In [ ]:
bnf_table.to_pickle('data/bnf_table_full.pkl')
bnf_table.to_pickle('data/bnf_table_summaries.pkl')